In [15]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests

In [33]:
folder_path = './CWE'
cleaned = './cleaned'

filtered = [
    'CWE ID', 'func_before', 'C3_Explanation_of_Vulnerability_In_Context', 'C5_Explanation_Vulnerability_Fixed_In_Context' ]

rename_col = {
    'func_before': 'vulnerable_code', 
    'C3_Explanation_of_Vulnerability_In_Context': 'Explanation_of_Vulnerability_In_Context', 
    'C5_Explanation_Vulnerability_Fixed_In_Context': 'Explanation_Vulnerability_Fixed_In_Context', 
}

original_sum = 0
sum = 0
less_than_three = []
drop_empty = 0

template_df = pd.DataFrame(columns=filtered)

for cwe in os.listdir(folder_path):
    if cwe.endswith('.csv'):
        file_path = os.path.join(folder_path, cwe)
        df = pd.read_csv(file_path)
        initial = len(df)
        original_sum += initial

        if len(df) < 3:
            less_than_three.append([cwe, len(df)])

        df = df.reindex(columns=filtered, fill_value='')
        df = pd.concat([template_df, df], ignore_index=True)
        df.rename(columns=rename_col, inplace=True)
        if 'CWE ID' in df.columns:
            df.rename(columns={'CWE ID': 'CWE_ID'}, inplace=True)
        cwe_id = cwe.replace('.csv', '')
        df['CWE_ID'] = cwe_id

        before = len(df)
        df = df[
            ~(df['Explanation_Vulnerability_Fixed_In_Context'].isin(['', 'nan']) | df['Explanation_Vulnerability_Fixed_In_Context'].isna()) &
            ~(df['Explanation_of_Vulnerability_In_Context'].isin(['', 'nan']) | df['Explanation_of_Vulnerability_In_Context'].isna())
        ]

        df = df.dropna(subset=['Explanation_Vulnerability_Fixed_In_Context', 'Explanation_of_Vulnerability_In_Context'])
        drop_empty += before - len(df)
        # print(cwe_id)
        # print(df)

        df = df.dropna(subset=['vulnerable_code', 'CWE_ID'])
        if len(df)!=initial:
            print(cwe, 'is filtered to', len(df), 'rows, original:', initial)

        df = df.reset_index(drop=True)
        sum += len(df)

        cleaned_file_path = os.path.join(cleaned, cwe)
        df.to_csv(cleaned_file_path, index=False)

CWE-20.csv is filtered to 217 rows, original: 233
CWE-835.csv is filtered to 16 rows, original: 19
CWE-200.csv is filtered to 127 rows, original: 130
CWE-416.csv is filtered to 77 rows, original: 83
CWE-399.csv is filtered to 109 rows, original: 110
CWE-415.csv is filtered to 18 rows, original: 19
CWE-94.csv is filtered to 2 rows, original: 3
CWE-476.csv is filtered to 78 rows, original: 84
CWE-674.csv is filtered to 0 rows, original: 2
CWE-369.csv is filtered to 8 rows, original: 18
CWE-787.csv is filtered to 37 rows, original: 43
CWE-190.csv is filtered to 56 rows, original: 60


In [34]:
drop_empty

59

In [18]:
less_than_three

[['CWE-404.csv', 1],
 ['CWE-361.csv', 1],
 ['CWE-601.csv', 1],
 ['CWE-172.csv', 1],
 ['CWE-824.csv', 2],
 ['CWE-664.csv', 1],
 ['CWE-909.csv', 1],
 ['CWE-1021.csv', 1],
 ['CWE-918.csv', 2],
 ['CWE-311.csv', 1],
 ['CWE-502.csv', 1],
 ['CWE-674.csv', 2],
 ['CWE-89.csv', 2],
 ['CWE-532.csv', 2],
 ['CWE-134.csv', 1],
 ['CWE-295.csv', 2],
 ['CWE-732.csv', 2],
 ['CWE-74.csv', 1],
 ['CWE-327.csv', 2],
 ['CWE-290.csv', 1],
 ['CWE-668.csv', 1],
 ['CWE-331.csv', 1],
 ['CWE-682.csv', 2],
 ['CWE-209.csv', 1],
 ['CWE-16.csv', 1],
 ['CWE-354.csv', 2],
 ['CWE-755.csv', 1],
 ['CWE-345.csv', 2],
 ['CWE-352.csv', 1]]

In [38]:
sum

1196

In [39]:
original_sum

1255

In [19]:
filenames = ["train.csv", "test.csv", "val.csv"]
path = "./result"
temp = pd.DataFrame()
filtered = [
    'CWE ID', 'func_before', 'C3_Explanation_of_Vulnerability_In_Context', 'C5_Explanation_Vulnerability_Fixed_In_Context' ]

for filename in filenames:
    full_path = os.path.join(path, filename)
    if not os.path.exists(full_path):
        temp_df = pd.DataFrame(columns=filtered)
        temp_df.to_csv(full_path, index=False)
        print(f"{filename} created")
    else:
        print(f"{filename} already exists.")

train.csv created
test.csv created
val.csv created


In [25]:
folder_path = './cleaned'
filenames = ["train.csv", "test.csv", "val.csv"]
path = "./result"
exported = 0

dataframes = {filename: pd.DataFrame() for filename in filenames}
for cwe in os.listdir(folder_path):
    if cwe.endswith('.csv'):
        file_path = os.path.join(folder_path, cwe)
        df = pd.read_csv(file_path)

        chunk_size = len(df) // len(filenames)
        remainder = len(df) % len(filenames)

        for i, filename in enumerate(filenames):
            start_idx = i * chunk_size
            end_idx = start_idx + chunk_size + (1 if i < remainder else 0)
            chunk = df.iloc[start_idx:end_idx]
            dataframes[filename] = pd.concat([dataframes[filename], chunk], ignore_index=True)

for filename, dataframe in dataframes.items():
    exported += len(dataframe)
    full_path = os.path.join(path, filename)
    dataframe.to_csv(full_path, index=False)


Result Check (First dataset)

In [33]:
val = pd.read_csv('./result/val.csv')
test = pd.read_csv('./result/test.csv')
train = pd.read_csv('./result/train.csv')

In [37]:
print('unique CWE:', val['CWE_ID'].nunique())
print('number of entries:', len(val))

unique CWE: 39
number of entries: 391


In [38]:
print('unique CWE:', test['CWE_ID'].nunique())
print('number of entries:', len(test))

unique CWE: 50
number of entries: 416


In [39]:
print('unique CWE:', train['CWE_ID'].nunique())
print('number of entries:', len(train))

unique CWE: 68
number of entries: 448


In [40]:
train

,CWE_ID,vulnerable_code,fixed_code,Description_of_Functionality_In_Context,Description_of_Functionality_Generic,Code_Change,Explanation_of_Vulnerability_In_Context,Explanation_of_Vulnerability_Generic,Explanation_Vulnerability_Fixed_In_Context,Explanation_Vulnerability_Fixed_Generic
0,CWE-834,"static int read_tfra(MOVContext *mov, AVIOCont...","static int read_tfra(MOVContext *mov, AVIOCont...",The `read_tfra` function processes a 'tfra' (T...,This function reads and processes a specific t...,The updated code introduces a check for end-of...,The vulnerability was caused by the absence of...,The vulnerability was caused by the lack of a ...,The vulnerability was fixed by introducing an ...,The vulnerability was fixed by adding a check ...
1,CWE-834,static MagickBooleanType ReadPSDLayersInternal...,static MagickBooleanType ReadPSDLayersInternal...,The `ReadPSDLayersInternal` function processes...,The function processes and extracts layer info...,The primary change in the code involves the ad...,The vulnerability was caused by the absence of...,The vulnerability was caused by a lack of prop...,The vulnerability was fixed by adding an end-o...,The vulnerability was fixed by introducing a c...
2,CWE-834,static Image *ReadPSImage(const ImageInfo *ima...,static Image *ReadPSImage(const ImageInfo *ima...,The `ReadPSImage` function is designed to read...,The function processes PostScript image files ...,The primary change in the code involves the ad...,The vulnerability was caused by the absence of...,The vulnerability was caused by a failure to i...,The vulnerability was fixed by adding a check ...,The vulnerability was fixed by implementing a ...
3,CWE-388,"gs_call_interp(i_ctx_t **pi_ctx_p, ref * pref,...","gs_call_interp(i_ctx_t **pi_ctx_p, ref * pref,...",The `gs_call_interp` function is responsible f...,"The function interprets PostScript code, handl...","In the updated code, a modification was made t...",The vulnerability in the code was caused by in...,The vulnerability stemmed from inadequate inte...,The vulnerability was addressed by modifying t...,The vulnerability was mitigated by adjusting t...
4,CWE-388,static int cp2112_gpio_direction_input(struct ...,static int cp2112_gpio_direction_input(struct ...,This code defines a function `cp2112_gpio_dire...,The code function retrieves device data and HI...,"In the code, the only change made was in the r...",The vulnerability in the `cp2112_gpio_directio...,The vulnerability in the code stemmed from the...,The vulnerability in the `cp2112_gpio_directio...,The vulnerability was mitigated by adjusting t...
...,...,...,...,...,...,...,...,...,...,...
443,CWE-190,static int vfio_msi_enable(struct vfio_pci_dev...,static int vfio_msi_enable(struct vfio_pci_dev...,The `vfio_msi_enable` function enables MSI or ...,The code enables MSI or MSIX interrupts for a ...,"In the code after the change, the `kzalloc` fu...",The vulnerability in the original code stemmed...,The vulnerability in the original code was cau...,The vulnerability was addressed by replacing t...,The vulnerability was resolved by replacing th...
444,CWE-190,"choose_volume(struct archive_read *a, struct i...","choose_volume(struct archive_read *a, struct i...",The `choose_volume` function in the code selec...,The code determines the appropriate volume des...,"In the code after the change, the modification...",The vulnerability in the code was caused by an...,The vulnerability in the code was caused by an...,The vulnerability was fixed by addressing the ...,The vulnerability was fixed by ensuring consis...
445,CWE-347,DynamicMetadataProvider::DynamicMetadataProvi...,DynamicMetadataProvider::DynamicMetadataProvi...,The `DynamicMetadataProvider` class is a const...,The code initializes various settings and para...,"In the code after the change, the main modific...",The vulnerability in the DynamicMetadataProvid...,The vulnerability in the DynamicMetadataProvid...,The vulnerability in the DynamicMetadataProvid...,The vulnerabil

In [41]:
test

,CWE_ID,vulnerable_code,fixed_code,Description_of_Functionality_In_Context,Description_of_Functionality_Generic,Code_Change,Explanation_of_Vulnerability_In_Context,Explanation_of_Vulnerability_Generic,Explanation_Vulnerability_Fixed_In_Context,Explanation_Vulnerability_Fixed_Generic
0,CWE-834,static Image *ReadPSImage(const ImageInfo *ima...,static Image *ReadPSImage(const ImageInfo *ima...,The `ReadPSImage` function is designed to read...,The function processes PostScript image files ...,The primary change in the code involves the ad...,The vulnerability was caused by the absence of...,The vulnerability was caused by a failure to i...,The vulnerability was fixed by adding a check ...,The vulnerability was fixed by implementing a ...
1,CWE-834,static int nsv_parse_NSVf_header(AVFormatConte...,static int nsv_parse_NSVf_header(AVFormatConte...,The provided code defines a function `nsv_pars...,The code implements a function that processes ...,The modified code introduces an additional err...,The vulnerability was caused by the absence of...,The vulnerability was caused by a lack of prop...,The vulnerability was fixed by introducing an ...,The vulnerability was fixed by adding a check ...
2,CWE-834,static int cine_read_header(AVFormatContext *a...,static int cine_read_header(AVFormatContext *a...,The `cine_read_header` function is responsible...,The function is designed to read and interpret...,The updated code introduces a check for end-of...,The vulnerability was caused by the absence of...,The vulnerability was caused by a lack of an e...,The vulnerability was fixed by adding an end-o...,The vulnerability was fixed by introducing a c...
3,CWE-388,static int cp2112_gpio_direction_input(struct ...,static int cp2112_gpio_direction_input(struct ...,This code defines a function `cp2112_gpio_dire...,The code function retrieves device data and HI...,"In the code, the only change made was in the r...",The vulnerability in the `cp2112_gpio_directio...,The vulnerability in the code stemmed from the...,The vulnerability in the `cp2112_gpio_directio...,The vulnerability was mitigated by adjusting t...
4,CWE-20,"static int dccp_packet(struct nf_conn *ct, con...","static int dccp_packet(struct nf_conn *ct, con...",The code defines the functionality of handling...,The code manages the states and roles of DCCP ...,The only change made in the code is in the lin...,The vulnerability was caused by the incorrect ...,The vulnerability stemmed from mishandling the...,The vulnerability was fixed by correcting the ...,The vulnerability was addressed by ensuring th...
...,...,...,...,...,...,...,...,...,...,...
411,CWE-190,static int b_unpack (lua_State *L) {\n Header...,static int b_unpack (lua_State *L) {\n Header...,The code defines a function `b_unpack` in Lua ...,The code defines a function that unpacks binar...,The change made in the code involves adjusting...,The vulnerability stemmed from an Integer Over...,The vulnerability was caused by an Integer Ove...,The vulnerability was fixed by adjusting the c...,The vulnerability was addressed by modifying t...
412,CWE-190,"mrb_vm_exec(mrb_state *mrb, struct RProc *proc...","mrb_vm_exec(mrb_state *mrb, struct RProc *proc...",The provided code is a function that executes ...,The code implements a virtual machine that exe...,The changes made to the code primarily involve...,The cause of the vulnerability was an integer ...,The vulnerability was caused by inadequate che...,The vulnerability was fixed by adding checks i...,The vulnerability was fixed by implementing ad...
413,CWE-190,static BOOL gdi_Bitmap_Decompress(rdpContext* ...,static BOOL gdi_Bitmap_Decompress(rdpContext* ...,The `gdi_Bitmap_Decompress` function is respon...,The function allocates memory for the decompre...,"In the updated code, a check was added to ensu...",The vulnerability stemmed from the lack of pro...,The vulnerability arose due to inadequate vali...,The vulnerability was addressed by introducing...,The vulnerability was mitig

In [42]:
val

,CWE_ID,vulnerable_code,fixed_code,Description_of_Functionality_In_Context,Description_of_Functionality_Generic,Code_Change,Explanation_of_Vulnerability_In_Context,Explanation_of_Vulnerability_Generic,Explanation_Vulnerability_Fixed_In_Context,Explanation_Vulnerability_Fixed_Generic
0,CWE-834,static int cine_read_header(AVFormatContext *a...,static int cine_read_header(AVFormatContext *a...,The `cine_read_header` function is responsible...,The function is designed to read and interpret...,The updated code introduces a check for end-of...,The vulnerability was caused by the absence of...,The vulnerability was caused by a lack of an e...,The vulnerability was fixed by adding an end-o...,The vulnerability was fixed by introducing a c...
1,CWE-834,static av_cold int rl2_read_header(AVFormatCon...,static av_cold int rl2_read_header(AVFormatCon...,The `rl2_read_header` function is responsible ...,The function is designed to read the header of...,The updated code introduces checks for end-of-...,The vulnerability was caused by the absence of...,The vulnerability was caused by the lack of ch...,The vulnerability was fixed by introducing che...,The vulnerability was fixed by adding checks t...
2,CWE-388,"vc4_get_bcl(struct drm_device *dev, struct vc4...","vc4_get_bcl(struct drm_device *dev, struct vc4...",The `vc4_get_bcl` function is responsible for ...,"The code function calculates offsets, checks f...","In the updated code, the change made was in th...",The vulnerability in the `vc4_get_bcl` functio...,The vulnerability in the code stemmed from the...,The vulnerability in the `vc4_get_bcl` functio...,The vulnerability in the code was remedied by ...
3,CWE-20,static opj_bool pi_next_cprl(opj_pi_iterator_t...,static opj_bool pi_next_cprl(opj_pi_iterator_t...,The code defines the function `pi_next_cprl` w...,"The code iterates through components, resoluti...","In the updated code, a check was added to prev...",The vulnerability in the functions `pi_next_lr...,The vulnerability in the affected functions of...,The vulnerability in the OpenJPEG library was ...,The vulnerability in the OpenJPEG library was ...
4,CWE-20,static opj_bool pi_next_lrcp(opj_pi_iterator_...,static opj_bool pi_next_lrcp(opj_pi_iterator_...,The provided code defines a function `pi_next_...,The code defines a function that iterates thro...,"In the updated code, a check was added to prev...",The vulnerability in the code was caused by ou...,The vulnerability in the code stemmed from ina...,The vulnerability was addressed by adding a ch...,The vulnerability was resolved by introducing ...
...,...,...,...,...,...,...,...,...,...,...
386,CWE-190,OperationID FileSystemOperationRunner::BeginOp...,OperationID FileSystemOperationRunner::BeginOp...,This code defines a method `BeginOperation` wi...,This code snippet defines a method within a cl...,"In the updated code, the change made was in th...",The vulnerability stemmed from an integer over...,The vulnerability was caused by the potential ...,The vulnerability was addressed by changing th...,The vulnerability was mitigated by changing th...
387,CWE-190,status_t BnSoundTriggerHwService::onTransact(\...,status_t BnSoundTriggerHwService::onTransact(\...,The code defines the `onTransact` function for...,The code defines a function that handles diffe...,"In the updated code, a check was added to limi...",The vulnerability stemmed from a lack of prope...,The vulnerability was caused by a lack of vali...,"In the updated code, the vulnerability was add...",The vulnerability was mitigated by introducing...
388,CWE-190,AMediaCodecCryptoInfo *AMediaCodecCryptoInfo_n...,AMediaCodecCryptoInfo *AMediaCodecCryptoInfo_n...,The `AMediaCodecCryptoInfo_new` function creat...,The code defines a function that creates a new...,"In the updated code, the calculation of the `c...",The vulnerability in the code was caused by an...,The vulnerability stemmed from an integer over...,The vulnerability was addressed by implementin...,The vulnerability 

In [15]:
train = pd.read_csv('./dataset/train.csv')
train['Explanation_of_Vulnerability_In_Context'].isna().sum()

18

In [16]:
test = pd.read_csv('./dataset/test.csv')
test['Explanation_of_Vulnerability_In_Context'].isna().sum()

16

In [17]:
val = pd.read_csv('./dataset/val.csv')
val['Explanation_of_Vulnerability_In_Context'].isna().sum()

19

switch to 80 - 10 - 10 distribution

In [35]:
folder_path = './cleaned'
filenames = ["train.csv", "test.csv", "val.csv"]
result_path = "./result"

dataframes = {filename: pd.DataFrame() for filename in filenames}

for cwe in os.listdir(folder_path):
    if cwe.endswith('.csv'):
        file_path = os.path.join(folder_path, cwe)
        df = pd.read_csv(file_path)
        
        # Ensure the column 'CWE_ID' exists
        df['CWE_ID'] = cwe.replace('.csv', '')

        # Assign the first row to the training set
        train_df = df.iloc[:1]
        remaining_df = df.iloc[1:]

        # Shuffle the remaining DataFrame to ensure random distribution
        remaining_df = remaining_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Split the remaining data into 80%, 10%, 10%
        n = len(remaining_df)
        train_end = int(n * 0.8) + 1  # +1 to include the first row already added to train
        test_end = train_end + int(n * 0.1)

        train_df = pd.concat([train_df, remaining_df.iloc[:train_end - 1]], ignore_index=True)
        test_df = remaining_df.iloc[train_end - 1:test_end]
        val_df = remaining_df.iloc[test_end:]

        dataframes['train.csv'] = pd.concat([dataframes['train.csv'], train_df], ignore_index=True)
        dataframes['test.csv'] = pd.concat([dataframes['test.csv'], test_df], ignore_index=True)
        dataframes['val.csv'] = pd.concat([dataframes['val.csv'], val_df], ignore_index=True)

# Save the dataframes to CSV files
for filename, dataframe in dataframes.items():
    full_path = os.path.join(result_path, filename)
    dataframe.to_csv(full_path, index=False)
    print(f"{filename} saved with {len(dataframe)} rows.")
    print(f"Unique CWE in {filename}: {dataframe['CWE_ID'].nunique()}")

train.csv saved with 943 rows.
Unique CWE in train.csv: 67
test.csv saved with 145 rows.
Unique CWE in test.csv: 49
val.csv saved with 108 rows.
Unique CWE in val.csv: 24


Result check (80 - 10 - 10)

In [36]:
val = pd.read_csv('./result/val.csv')
test = pd.read_csv('./result/test.csv')
train = pd.read_csv('./result/train.csv')

In [37]:
len(val) + len(test) + len(train)

1196

In [40]:
train

,CWE_ID,vulnerable_code,Explanation_of_Vulnerability_In_Context,Explanation_Vulnerability_Fixed_In_Context
0,CWE-834,"static int read_tfra(MOVContext *mov, AVIOCont...",The vulnerability was caused by the absence of...,The vulnerability was fixed by introducing an ...
1,CWE-834,static MagickBooleanType ReadPSDLayersInternal...,The vulnerability was caused by the absence of...,The vulnerability was fixed by adding an end-o...
2,CWE-834,static Image *ReadPSImage(const ImageInfo *ima...,The vulnerability was caused by the absence of...,The vulnerability was fixed by adding a check ...
3,CWE-834,static int mv_read_header(AVFormatContext *avc...,The vulnerability was caused by the absence of...,The vulnerability was fixed by adding a check ...
4,CWE-834,static int nsv_parse_NSVf_header(AVFormatConte...,The vulnerability was caused by the absence of...,The vulnerability was fixed by introducing an ...
...,...,...,...,...
938,CWE-347,DynamicMetadataProvider::DynamicMetadataProvi...,The vulnerability in the DynamicMetadataProvid...,The vulnerability in the DynamicMetadataProvid...
939,CWE-347,"static bigint *sig_verify(BI_CTX *ctx, const u...",The vulnerability in the code was caused by th...,The vulnerability was addressed by introducing...
940,CWE-347,static void DetectRunCleanup(DetectEngineThrea...,The vulnerability in the code was caused by a ...,The vulnerability in Suricata was addressed by...
941,CWE-347,DynamicMetadataProvider::DynamicMetadataProvi...,The vulnerability in the DynamicMetadataProvid...,The vulnerability in the DynamicMetadataProvid...


In [41]:
test

,CWE_ID,vulnerable_code,Explanation_of_Vulnerability_In_Context,Explanation_Vulnerability_Fixed_In_Context
0,CWE-834,static int cine_read_header(AVFormatContext *a...,The vulnerability was caused by the absence of...,The vulnerability was fixed by adding an end-o...
1,CWE-388,void processRequest(struct reqelem * req)\n{\n...,The vulnerability in the `processRequest` func...,The vulnerability in the MiniSSDPd code was ad...
2,CWE-20,int PDFiumEngine::GetVisiblePageIndex(FPDF_PA...,The vulnerability stemmed from directly iterat...,The vulnerability was addressed by creating a ...
3,CWE-20,"static float *get_window(vorb *f, int len)\n{\...",The vulnerability in the original code was cau...,The vulnerability was addressed by removing th...
4,CWE-20,static vpx_image_t *img_alloc_helper(vpx_image...,The vulnerability in the code was caused by in...,The vulnerability was fixed by adjusting the c...
...,...,...,...,...
140,CWE-190,ZEND_API void ZEND_FASTCALL _zend_hash_init(Ha...,The vulnerability stemmed from mishandling cas...,The vulnerability was addressed by adjusting t...
141,CWE-190,void ihevcd_parse_sei_payload(codec_t *ps_code...,The vulnerability in the code was caused by an...,The vulnerability was addressed by introducing...
142,CWE-190,"jas_matrix_t *jas_seq2d_create(int xstart, int...",The vulnerability in the code was caused by th...,The vulnerability was addressed by changing th...
143,CWE-190,int qcow2_snapshot_load_tmp(BlockDriverState *...,The vulnerability stemmed from multiple intege...,The vulnerability was addressed by adding a ch...


In [42]:
val

,CWE_ID,vulnerable_code,Explanation_of_Vulnerability_In_Context,Explanation_Vulnerability_Fixed_In_Context
0,CWE-834,static MagickBooleanType ReadPSDLayersInternal...,The cause of the vulnerability was the absence...,The vulnerability was fixed by introducing a c...
1,CWE-20,bool RenderFrameDevToolsAgentHost::AttachSessi...,The vulnerability was caused by the lack of pr...,The vulnerability was addressed by introducing...
2,CWE-20,"bool extractPages (const char *srcFileName, co...",The vulnerability stemmed from a format string...,The vulnerability was addressed by implementin...
3,CWE-20,int ssl_parse_certificate( ssl_context *ssl )\...,The vulnerability in the original code stemmed...,The vulnerability was fixed by replacing the g...
4,CWE-20,ModuleExport size_t RegisterMPCImage(void)\n{\...,The vulnerability in the code was caused by no...,The vulnerability was addressed by enabling se...
...,...,...,...,...
103,CWE-190,static void opj_get_encoding_parameters(const ...,The vulnerability in the function `opj_get_enc...,The vulnerability was fixed by introducing a m...
104,CWE-190,static int vfio_msi_enable(struct vfio_pci_dev...,The vulnerability in the original code stemmed...,The vulnerability was addressed by replacing t...
105,CWE-190,int URI_FUNC(ComposeQueryEngine)(URI_CHAR * de...,The vulnerability in the code stemmed from an ...,The vulnerability was addressed by introducing...
106,CWE-190,OperationID FileSystemOperationRunner::BeginOp...,The vulnerability stemmed from an integer over...,The vulnerability was addressed by changing th...
